# Importing the data
### setting comprehensible col names and right types

In [ ]:
# Importing all the necessary packages
import pandas as pd
import re
import numpy as np
from prophet import Prophet
import warnings

In [ ]:
# Ignore FutureWarning for deprecated 'T' frequency in Prophet
warnings.filterwarnings("ignore", category=FutureWarning, message="'T' is deprecated")

# Ignore PerformanceWarning from pandas
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [3]:
# Mix of different sources, mostly ESO
balancing_df = pd.read_csv("balancing_data.csv")
# Demand data only for GB
GB_demand_df = pd.read_csv("demand_load_data.csv")
# Generation data only for GB
GB_generation_df = pd.read_csv("generation_data.csv")
# the price dataframe only concerns EPEX (only prices from there)
EPEX_price_df = pd.read_csv("price_data.csv")

In [4]:
def rename_balancing_columns(df):
    # Define a dictionary for concise renaming
    rename_map = {
        'GMT Time': 'GMT Time',
        'System Price (ESO Outturn) - GB (£/MWh)': 'System_Price',
        'NIV Outturn (+ve long) - GB (MW)': 'NIV_Outturn',
        'BM Bid Acceptances (total) - GB (MW)': 'BM_Bid_Acceptances',
        'BM Offer Acceptances (total) - GB (MW)': 'BM_Offer_Acceptances',
        'Total BSAD Volume - Turn Up - GB (MW)': 'BSAD_Turn_Up',
        'Total BSAD Volume - Turn Down - GB (MW)': 'BSAD_Turn_Down',
        'Total BSAD Volume - Total - GB (MW)': 'BSAD_Total',
        'Intraday Volume (EPEX Outturn, APX, MID) - GB (MWh)': 'EPEX_Intraday_Volume'
    }
    
    # Apply the renaming map
    df = df.rename(columns=rename_map)

    # Force all the non datetime columns to numeric
    for column in df.columns:
        if column != 'GMT Time':  # Skip the 'GMT Time' column
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Apply the function to rename columns in balancing_df
balancing_df = rename_balancing_columns(balancing_df)

print("Final columns:")
print(balancing_df.columns.values)


Final columns:
['GMT Time' 'System_Price' 'NIV_Outturn' 'BM_Bid_Acceptances'
 'BM_Offer_Acceptances' 'BSAD_Turn_Up' 'BSAD_Turn_Down' 'BSAD_Total'
 'EPEX_Intraday_Volume']


In [5]:
def rename_demand_columns(df):
    """
    Rename columns for easier reference and convert non-datetime columns to numeric.
    """
    # Define a dictionary for concise renaming
    rename_map = {
        'GMT Time': 'GMT Time',
        'Loss of Load Probability - Latest - GB ()': 'Loss_of_Load_Prob',
        'Actual Total Load - GB (MW)': 'Total_Load',
        'Demand Outturn (ITSDO) - GB (MW)': 'Demand_Outturn'
    }
    
    # Apply the renaming map
    df = df.rename(columns=rename_map)

    # Force all the non-datetime columns to numeric
    for column in df.columns:
        if column != 'GMT Time':  # Skip the 'GMT Time' column
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Apply the renaming and filling functions
GB_demand_df = rename_demand_columns(GB_demand_df)


print("Final columns:")
print(GB_demand_df.columns.values)

Final columns:
['GMT Time' 'Loss_of_Load_Prob' 'Total_Load' 'Demand_Outturn']


In [6]:
def rename_columns_generation(df):
    # Define a function to clean each column name
    def clean_column_name(col):
        # Extract the generation type using regex
        match = re.search(r'Actual Aggregated Generation By Type - (.+?) - GB', col)
        if match:
            # Replace spaces with underscores for readability
            return match.group(1).replace(" ", "_")
        return col  # Return the column as is if no match is found

    # Rename columns using the clean_column_name function
    df.columns = [clean_column_name(col) for col in df.columns]
    for column in df.columns:
        if column != 'GMT Time':  # Skip the 'GMT Time' column
            df[column] = pd.to_numeric(df[column], errors='coerce')

    return df

# Apply the function to rename columns in generation_df
GB_generation_df = rename_columns_generation(GB_generation_df)


print("Final columns:")
print(GB_generation_df.columns.values)

Final columns:
['GMT Time' 'Biomass' 'Fossil_Gas' 'Fossil_Hard_Coal' 'Fossil_Oil'
 'Hydro_Pumped_Storage' 'Hydro_Run-of-River_and_Poundage' 'Nuclear'
 'Solar' 'Wind_Onshore' 'Wind_Offshore']


In [7]:
def rename_epex_columns(df):
    # Define a dictionary for manual renaming based on your desired column names
    rename_map = {
        'GMT Time': 'GMT Time',
        'Day Ahead Price (EPEX half-hourly, local) - GB (LC/MWh)': 'Day_Ahead_Price',
        'Intraday Price (EPEX Outturn, APX, MID) - GB (£/MWh)': 'Intraday_Price'
    }

    # Rename columns using the dictionary
    df = df.rename(columns=rename_map)
    for column in df.columns:
        if column != 'GMT Time':  # Skip the 'GMT Time' column
            df[column] = pd.to_numeric(df[column], errors='coerce')
    
    return df

# Apply the function to rename columns in EPEX_price_df
EPEX_price_df = rename_epex_columns(EPEX_price_df)


print("Final columns:")
print(EPEX_price_df.columns.values)

Final columns:
['GMT Time' 'Day_Ahead_Price' 'Intraday_Price']


In [8]:
# Set 'GMT Time' as index for each dataframe
balancing_df.set_index('GMT Time', inplace=True)
GB_demand_df.set_index('GMT Time', inplace=True)
GB_generation_df.set_index('GMT Time', inplace=True)
EPEX_price_df.set_index('GMT Time', inplace=True)

# Merge using index
merged_df = balancing_df.join([GB_demand_df, GB_generation_df, EPEX_price_df], how='inner')
# We put back the datetime column into the merged DF and rename it for practicality
merged_df.reset_index(inplace=True)
merged_df.rename(columns={'GMT Time': 'Datetime'}, inplace=True)

print("Merged columns:")
print(merged_df.columns.values)

Merged columns:
['Datetime' 'System_Price' 'NIV_Outturn' 'BM_Bid_Acceptances'
 'BM_Offer_Acceptances' 'BSAD_Turn_Up' 'BSAD_Turn_Down' 'BSAD_Total'
 'EPEX_Intraday_Volume' 'Loss_of_Load_Prob' 'Total_Load' 'Demand_Outturn'
 'Biomass' 'Fossil_Gas' 'Fossil_Hard_Coal' 'Fossil_Oil'
 'Hydro_Pumped_Storage' 'Hydro_Run-of-River_and_Poundage' 'Nuclear'
 'Solar' 'Wind_Onshore' 'Wind_Offshore' 'Day_Ahead_Price' 'Intraday_Price']


# User Defined Functions

In [ ]:
def calculate_fft(series, n_top_seasonalities, threshold_pc=0.02):
    """
    Calculate significant positive frequencies and their amplitudes using Fast Fourier Transform (FFT),
    selecting the lower of 2% of the max amplitude or the top `n` frequencies.

    Parameters:
    - series (pd.Series): The input time series data.
    - n_top_seasonalities (int): The maximum number of significant frequencies to consider.
    - threshold_pc (float): Percentage (0 < threshold_pc <= 1) of the maximum amplitude to filter significant frequencies.

    Returns:
    - zip: A generator yielding (positive frequency, amplitude) for each significant frequency.
    """
    # Compute fast Fourier transform
    price_fft = np.fft.fft(series.dropna())

    # Get frequencies corresponding to FFT coefficients
    freqs = np.fft.fftfreq(len(price_fft), d=1/48)

    # Calculate amplitudes
    amplitudes = np.abs(price_fft)

    # Calculate the threshold based on 2% of the max amplitude
    threshold = threshold_pc * np.max(amplitudes)

    # Filter positive frequencies with amplitudes above threshold
    positive_indices = np.where((amplitudes > threshold) & (freqs > 0))
    positive_freqs = freqs[positive_indices]
    positive_amplitudes = amplitudes[positive_indices]

    # Sort by amplitude and select the lower of `n_top_seasonalities` or all significant frequencies
    sorted_indices = np.argsort(positive_amplitudes)[::-1]
    selected_indices = sorted_indices[:min(n_top_seasonalities, len(sorted_indices))]

    # Select the top frequencies and amplitudes
    significant_freqs = positive_freqs[selected_indices]
    significant_amplitudes = positive_amplitudes[selected_indices]

    return zip(significant_freqs, significant_amplitudes)


def prophet_predictions(series, freq_amp):
    """
    Generate predictions using Prophet with multiple seasonalities based on significant frequencies.

    Parameters:
    - series (pd.Series): The input time series data.
    - freq_amp (list of tuples): A list of (frequency, amplitude) pairs, where each frequency represents 
                                 a significant periodic component to be modeled as seasonality.

    Returns:
    - forecast (DataFrame): The forecasted values for the specified period, including trend and seasonal components.
    """
    # Prepare data for Prophet
    df = pd.DataFrame({'ds': series.index, 'y': series})
    model = Prophet()

    # Adding seasonalities based on significant frequencies
    for freq, amp in freq_amp:
        if freq != 0:  # Ignore the DC component
            period_in_days = 1 / freq
            seasonality_name = f"seasonal_freq_{freq:.4f}"
            fourier_order = 5 if period_in_days <= 1 else (10 if period_in_days <= 7 else 20)
            model.add_seasonality(name=seasonality_name, period=period_in_days, fourier_order=fourier_order)

    model.fit(df)
    future = model.make_future_dataframe(periods=48, freq='30T')
    forecast = model.predict(future)

    return forecast.set_index('ds')['yhat']


# Function to find the maximum number of consecutive NaNs filled in a column
# As Angelica Asked
def max_consecutive_nans_filled(df, column):
    """

    This function calculates and returns the maximum number 
    of consecutive NaNs in a column that is to be filled

    """
    # Identify consecutive NaNs
    na_groups = df[column].isna().astype(int).groupby(df[column].notna().cumsum()).sum()
    # Get the maximum number of consecutive NaNs that would be interpolated
    max_consecutive_nans = na_groups.max()
    nans_before = df[column].isna().sum()

    print(f"NaNs in {column}: {nans_before}")
    print(f"Max consecutive NaNs filled for '{column}': {max_consecutive_nans}")
    return


def fill_nans_with_prophet(series):
    """
    Fills NaNs in the original time series data using predictions from the Prophet model.

    Parameters:
    - series (pd.Series): The input time series data with potential NaNs.

    Returns:
    - pd.Series: The time series with NaNs filled using Prophet predictions.
    """
    if series.isna().any():
        print("NaN detected, proceeding to fill...")
        
        # Step 1: Calculate significant frequencies
        freq_amp = calculate_fft(series, n_top_seasonalities=12, threshold_pc=0.02)
        print("Frequencies and amplitudes calculated:", freq_amp)

        # Step 2: Generate Prophet predictions
        predictions = prophet_predictions(series, freq_amp)
        print(f"Predictions length: {len(predictions)}, Series length: {len(series)}")
        
        # Step 3: Trim predictions to match series index
        predictions = predictions.iloc[:len(series)]
        predictions.index = series.index  # Align indices

        # Step 4: Fill NaNs in the series
        filled_series = series.combine_first(predictions)
        print("NaNs filled in the series.")
        
        return filled_series
    else:
        print("No NaNs detected. Returning original series.")
        return series


def process_dataframe(df):
    """
    Iterates over each column of the DataFrame, applying Prophet-based NaN filling where applicable.

    Parameters:
    - df (pd.DataFrame): The DataFrame with multiple time series columns, potentially containing NaNs.

    Returns:
    - pd.DataFrame: The DataFrame with NaNs filled where possible.
    """
    for column in df.columns:
        # Check if the column data type is numeric (Prophet requires numeric types)
        if column != "Datetime":
            print(f"Processing column: {column}")
            df[column] = fill_nans_with_prophet(df[column])
        else:
            print(f"Skipping column: {column} (non-numeric data)")
    return df

## Filling in the NaNs

### So First, we try to fill in the columns that can be filled using other columns
### NIV_Outturn = - (BM_Bid_Acceptances + BM_Offer_Acceptances) 
### and 
### BSAD_Total = BSAD_Turn_Down + BSAD_Turn_Up

In [11]:
# Replace the "No Data Available" by 0s in the BSAD columns where applicable
# if all three are missing we just let them be replaced by NaNs

# Replace "No Data Available" in "BSAD_Turn_Up" with 0 if "BSAD_Total" is equal to other column
merged_df.loc[(merged_df["BSAD_Turn_Up"].isna()) & (merged_df["BSAD_Total"] == merged_df["BSAD_Turn_Down"]), "BSAD_Turn_Up"] = 0

# Replace "No Data Available" in "BSAD_Turn_Down" with 0 if "BSAD_Total" is equal to other column
merged_df.loc[(merged_df["BSAD_Turn_Down"].isna()) & (merged_df["BSAD_Total"] == merged_df["BSAD_Turn_Up"]), "BSAD_Turn_Down"] = 0    

# Replace 'NIV_Outturn' with NaN if both 'BM_Bid_Acceptances' and 'BM_Offer_Acceptances' are NaN and 'NIV_Outturn' is 0
merged_df.loc[(merged_df['NIV_Outturn'] == 0) & merged_df['BM_Bid_Acceptances'].isna() & merged_df['BM_Offer_Acceptances'].isna(), 'NIV_Outturn'] = np.nan

# Replace 'NIV_Outturn' with the negative of the sum of 'BM_Offer_Acceptances' and 'BM_Bid_Acceptances' 
# if 'NIV_Outturn' is zero and neither of the other two columns contains NaN
merged_df.loc[(merged_df['NIV_Outturn'] == 0) & merged_df['BM_Offer_Acceptances'].notna() & merged_df['BM_Bid_Acceptances'].notna(), 'NIV_Outturn'] = -(merged_df['BM_Offer_Acceptances'] + merged_df['BM_Bid_Acceptances'])

# Extrapolate 'BM_Bid_Acceptances' with condition to set both columns to NaN if bid check fails
bid_values = -merged_df['NIV_Outturn'] - merged_df['BM_Offer_Acceptances']
merged_df.loc[merged_df['BM_Bid_Acceptances'].isna() & merged_df['NIV_Outturn'].notna(), 'BM_Bid_Acceptances'] = bid_values.where(bid_values <= 0)
merged_df.loc[merged_df['BM_Bid_Acceptances'].isna(), 'BM_Offer_Acceptances'] = np.nan

# Extrapolate 'BM_Offer_Acceptances' with condition to set both columns to NaN if offer check fails
offer_values = -merged_df['NIV_Outturn'] - merged_df['BM_Bid_Acceptances']
merged_df.loc[merged_df['BM_Offer_Acceptances'].isna() & merged_df['NIV_Outturn'].notna(), 'BM_Offer_Acceptances'] = offer_values.where(offer_values >= 0)
merged_df.loc[merged_df['BM_Offer_Acceptances'].isna(), 'BM_Bid_Acceptances'] = np.nan

### Mark the rows where there are missing values for each variable

In [12]:
for column in merged_df.columns:
    merged_df[f'{column}_missing'] = merged_df[column].isnull().astype(int)

In [13]:
merged_df = merged_df.set_index("Datetime")

### Then, we fill the other NaNs using Prophet

In [ ]:
# Process the DataFrame
merged_df = process_dataframe(merged_df)

NaN detected, proceeding to fill...
Frequencies and amplitudes calculated: <zip object at 0x0000014F68B3DC40>


18:10:53 - cmdstanpy - INFO - Chain [1] start processing
18:17:37 - cmdstanpy - INFO - Chain [1] done processing


Predictions length: 118368, Series length: 118320
NaNs filled in the series.
NaN detected, proceeding to fill...
Frequencies and amplitudes calculated: <zip object at 0x0000014F69616280>


18:18:55 - cmdstanpy - INFO - Chain [1] start processing
18:25:15 - cmdstanpy - INFO - Chain [1] done processing


Predictions length: 118368, Series length: 118320
NaNs filled in the series.
NaN detected, proceeding to fill...
Frequencies and amplitudes calculated: <zip object at 0x0000014F7C0EB380>


18:26:35 - cmdstanpy - INFO - Chain [1] start processing
18:39:29 - cmdstanpy - INFO - Chain [1] done processing


Predictions length: 118368, Series length: 118320
NaNs filled in the series.


### Save the filled df in order to retrieve it later for the models

In [ ]:
merged_df.reset_index(inplace=True)
merged_df.to_csv("merged_df_Combined.csv", index=False)